In [9]:
import pandas as pd
from tqdm import tqdm
import networkx as nx

In [10]:
a_q=pd.read_csv('a2q',sep=' ', names=['u','v','t'])  #answers to questions
c_a=pd.read_csv('c2a',sep=' ', names=['u','v','t'])  #comments to answers
c_q=pd.read_csv('c2q',sep=' ', names=['u','v','t'])  #comments to questions

In [11]:
print(a_q.shape,c_a.shape,c_q.shape)

(17823525, 3) (25405374, 3) (20268151, 3)


In [12]:
#change time to hours
a_q['t'] = a_q['t'].div(3600)
c_a['t'] = c_a['t'].div(3600)
c_q['t'] = c_q['t'].div(3600)

#add type column to dataframes
a_q['type'] = 'a_q'
c_a['type'] = 'c_a'
c_q['type'] = 'c_q'

In [13]:
def time_maxmin(data,min,max): #Retrieve maximum and minimum timestamps
    m = data['t'].min()
    M = data['t'].max()
    if m < min:
        min = m 
    if M > max:
        max = M
    return min, max

In [14]:
def create_graph(data1,time_window,self_loops=0):
    #remove rows with u=v
    if self_loops == 0: 
        data1 = data1[data1['u'] != data1['v']]
    
    #remove rows not having time in given time interval
    data1 = data1[data1['t'].between(time_window[0], time_window[1])]
    
    G = nx.MultiDiGraph()
    G = nx.from_pandas_edgelist(data1, 'u', 'v', ['t', 'type'])
   
    return G

In [15]:
m=10**(10)
M=0
m, M = time_maxmin(a_q, m, M)
m, M = time_maxmin(c_a, m, M)
m, M = time_maxmin(c_q, m, M)
print(m, M)

338213.2991666667 404798.1744444444


In [16]:
#obviously choose a time window that's inside (m,M)
time_window = (M/2,M/2+1000)
A = create_graph(a_q,time_window)
B = create_graph(c_a,time_window)
C = create_graph(c_q,time_window)

In [17]:
#merge the graphs
G = nx.compose(A,B)
G = nx.compose(G,C)